# Clean the data after correcting

## 1. Train a tokenizer with the cleaned corpus

The output folder will be the same, so the previous tokenizer will be overwritten.

In [1]:
import pandas as pd
from transformers import AutoTokenizer
import json
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
VOCAB_SIZE = 52000
TRAINING_BATCH_SIZE = 1000
HF_CHECKPOINT = "dccuchile/bert-base-spanish-wwm-cased"

pretrained_tokenizer = AutoTokenizer.from_pretrained(HF_CHECKPOINT)
df = pd.read_csv("../data/old-spanish-corpus-cleaned.tsv", sep='\t')

training_corpus = (
    df.loc[i:i+TRAINING_BATCH_SIZE, "text"].astype(str) 
    for i in range(0, len(df), TRAINING_BATCH_SIZE)
)

tokenizer = pretrained_tokenizer.train_new_from_iterator(training_corpus, VOCAB_SIZE)

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\santi\AppData\Local\Temp\ipykernel_13636\2841662301.py:6: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/old-spanish-corpus-cleaned.tsv", sep='\t')


In [3]:
example = df.loc[5, "text"]
print("BEFORE:", pretrained_tokenizer.tokenize(example)[:20])
print("AFTER:", tokenizer.tokenize(example)[:20])
tokenizer.save_pretrained("../output/tokenizer")

BEFORE: ['CON', '##QU', '##ISTA', '4', '*', '*', 'cu', '##tar', 'lo', 'que', 'fuese', 'de', 'su', 'mayor', 'agrado', ',', 'sin', 'dis', '"', 'curr']
AFTER: ['CONQUISTA', '4', '*', '*', 'cu', '##tar', 'lo', 'que', 'fuese', 'de', 'su', 'mayor', 'agrado', ',', 'sin', 'dis', '"', 'cur', '##rir', 'en']


('../output/tokenizer\\tokenizer_config.json',
 '../output/tokenizer\\special_tokens_map.json',
 '../output/tokenizer\\vocab.txt',
 '../output/tokenizer\\added_tokens.json',
 '../output/tokenizer\\tokenizer.json')

## 2. Chunk the texts that have more than 512 tokens

512 is the maximum number of tokens that can be processed by BERT-like models.

### 1. Chunk all the texts that have more than 512 tokens

In [4]:
MAX_TOKENS_LENGTH = 256 # 512/2, a number much below 512 so that it will still fit for different tokenizers

def chunk(text):
    chunks = []
    sentences = nltk.sent_tokenize(text) # text.split('.'), but enhanced
    current_chunk = sentences[0]
    for sentence in sentences[1:]:
        new_chunk_tks = len(tokenizer(f"{current_chunk} {sentence}")['input_ids'])
        if (new_chunk_tks) > (MAX_TOKENS_LENGTH-2):
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
            current_chunk = f"{current_chunk} {sentence}"
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

In [5]:
idx_max = dict()
for i, sentence in enumerate(df['text'].astype(str)):
    num_tokens = len(tokenizer(sentence)['input_ids'])
    if num_tokens > MAX_TOKENS_LENGTH:
        chunks = chunk(sentence)
        idx_max[i] = chunks
    if i % 100000 == 0:
        print(f"Chunked {i}/{len(df)} ({i/len(df):.2%})")

print("Finished 100%")

Chunked 0/734790 (0.00%)


Token indices sequence length is longer than the specified maximum sequence length for this model (694 > 512). Running this sequence through the model will result in indexing errors


Chunked 100000/734790 (13.61%)
Chunked 200000/734790 (27.22%)
Chunked 300000/734790 (40.83%)
Chunked 400000/734790 (54.44%)
Chunked 500000/734790 (68.05%)
Chunked 600000/734790 (81.66%)
Chunked 700000/734790 (95.27%)
Finished 100%


In [6]:
assert df.loc[list(idx_max.keys())[0], "text"][:10] == idx_max[list(idx_max.keys())[0]][0][:10], "Texts do not coincide"

In [7]:
with open("../data/chunkedTexts.json", "w") as json_file:
    json.dump(idx_max, json_file, indent=4)

#### 2. Update the original DataFrame with the chunked texts

In [8]:
with open("../data/chunkedTexts.json", "r") as json_file:
    idx_max = json.load(json_file)
idx_max = {int(k): v for k, v in idx_max.items()}

In [9]:
original_length = len(df)
updated_length = len(df)+sum([len(i)-1 for i in idx_max.values()])
print(f"\nAdded {updated_length - original_length} chunks ({updated_length / original_length - 1:.2%})")


Added 536199 chunks (72.97%)


In [10]:
finaldf = []
for idx, row in df.iterrows():
    if idx in idx_max:
        cI = 0
        for t in idx_max[idx]:
            new_row = row.copy()
            new_row['text'] = t
            new_row['chunk_id'] = cI
            finaldf.append(new_row)
            cI += 1
    else:
        row['chunk_id'] = 0
        finaldf.append(row)
    if idx % 100000 == 0:
        print(f"Added {len(finaldf)}/{updated_length} ({len(finaldf)/updated_length:.2%})")

print("Finished 100%")
assert len(finaldf) == updated_length, f"Lengths must match, expected length to be {updated_length}, but was {len(finaldf)}"

Added 1/1270989 (0.00%)
Added 281901/1270989 (22.18%)
Added 528275/1270989 (41.56%)
Added 744909/1270989 (58.61%)
Added 918367/1270989 (72.26%)
Added 1022798/1270989 (80.47%)
Added 1126152/1270989 (88.60%)
Added 1234152/1270989 (97.10%)
Finished 100%


In [11]:
newcols = list(df.columns)
newcols.insert(3, "chunk_id")
finaldf = pd.DataFrame(finaldf, columns=newcols)
finaldf = finaldf.reset_index(drop=True)

finaldf

,source,source_id,source_text_id,chunk_id,title,date,place,text
0,The British Library,3436138,9,0,"Historia de la conquista de México, etc [With ...",1809,London,"HISTORIA DE LA CONQUISTA PE MÉXICO, POBLACIÓN ..."
1,The British Library,3436138,10,0,"Historia de la conquista de México, etc [With ...",1809,London,"This Work, as well as LAS FÁBULAS LITERARIAS, ..."
2,The British Library,3436138,11,0,"Historia de la conquista de México, etc [With ...",1809,London,"HISTORIA De la Conquista, población y Progreso..."
3,The British Library,3436138,12,0,"Historia de la conquista de México, etc [With ...",1809,London,"CONQUISTA quartel observando la batalla, y rec..."
4,The British Library,3436138,12,1,"Historia de la conquista de México, etc [With ...",1809,London,Pon deró con afectada seguridad el atrevimient...
...,...,...,...,...,...,...,...,...
1270984,Project Gutenberg,53294,916,0,Recuerdos Del Tiempo Viejo,1817-1893,?,Pero amo á Barcelona por tiranía de ley in...
1270985,Project Gutenberg,53294,917,0,Recuerdos Del Tiempo Viejo,1817-1893,?,Barcelona trabaja... y á su existencia el ...
1270986,Project Gutenberg,53294,919,0,Recuerdos Del Tiempo Viejo,1817-1893,?,Olvidaba que entre ambas hay diferencia: n...
1270987,Project Gutenberg,53294,920,0,Recuerdos Del Tiempo Viejo,1817-1893,?,La diferencia es esta: pero es preciso que...


In [12]:
finaldf.to_csv("../data/old-spanish-corpus-chunked.tsv", sep="\t", index=False)

In [13]:
total_tokens = finaldf['text'].apply(lambda x: len(tokenizer.tokenize(x))).sum()
total_words = finaldf['text'].apply(lambda x: len(x.split())).sum()

In [14]:
print(f"New dataset has:\n\ttokens: {total_tokens:,}\n\twords: {total_words:,}\n\ttexts: {len(finaldf):,}")

New dataset has:
	tokens: 183,812,665
	words: 142,160,143
	texts: 1,270,989


In [15]:
latamp = finaldf[finaldf['source'] == 'LatamXIX']
total_tokens = latamp['text'].apply(lambda x: len(tokenizer.tokenize(x))).sum()
total_words = latamp['text'].apply(lambda x: len(x.split())).sum()

In [18]:
print(f"Latin-American portion of dataset has:\n\ttokens: {total_tokens:,}\n\twords: {total_words:,}\n\ttexts: {len(latamp):,}")

Latin-American portion of dataset has:
	tokens: 29,319,750
	words: 22,008,851
	texts: 158,283
